In [1]:
#install import stuff, you need to login and give google permissions twice
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!pip install pandas==0.22

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 113597 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [2]:
#mount google drive
%cd
!mkdir -p drive
!google-drive-ocamlfuse drive

/root


In [3]:
#navigating your google drive and saving files inside.
%cd
%ls
%cd drive/
%mkdir github_st5220
%cd github_st5220
%rm -r ST5220
!git clone https://github.com/leexa90/ST5220.git
#download model parameters
%ls -lh */
%ls -lh
%cd ST5220
%ls -lh

/root
drive/
/root/drive
/root/drive/github_st5220
rm: cannot remove 'ST5220': No such file or directory
Cloning into 'ST5220'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 8 (delta 0), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.
Checking out files: 100% (3/3), done.
total 62M
-rw-r--r-- 1 root root  56M Feb 12 07:43 2015.csv.zip
-rw-r--r-- 1 root root 6.7M Feb 12 07:43 codebook15_llcp.pdf
-rw-r--r-- 1 root root   75 Feb 12 07:43 README.md
total 4.0K
drwxr-xr-x 2 root root 4.0K Feb 12 07:40 ST5220/
/root/drive/github_st5220/ST5220
total 62M
-rw-r--r-- 1 root root  56M Feb 12 07:43 2015.csv.zip
-rw-r--r-- 1 root root 6.7M Feb 12 07:43 codebook15_llcp.pdf
-rw-r--r-- 1 root root   75 Feb 12 07:43 README.md


# **0. Figure out data types to efficiently load to memory**

In [4]:
import pandas as pd
from collections import Counter
import numpy as np
import gc
from itertools import combinations
#small dataset to figure out data types to load them more efficiently to memory
data= pd.read_csv('2015.csv.zip', compression='zip',sep=',',nrows=300)
dtypes = {}
for i in data.keys():
    if np.dtype(data[i]) != object:
        dtypes[i] = np.float32
    else: print i
   

IDATE
IMONTH
IDAY
IYEAR
PCDMDECN
EXACTOT1
EXACTOT2


# **1. Read entire dataset 400k datapoints, dummy variable for missing values in original dataset= -999**

In [0]:
data= pd.read_csv('2015.csv.zip', compression='zip',sep=',',nrows=500000,dtype=dtypes)


# Dummy value -999 for missing values
monthDay_to_int ={}
for i in range(32):
    str_i = str(i)
    if len(str_i) ==1:
        str_i = '0'+str_i
    monthDay_to_int["b'%s'" %str_i] = i
   
# changing datatype of day and month of survey
data['IMONTH']=data['IMONTH'].map(monthDay_to_int).astype(np.int32)
data['IDAY']=data['IDAY'].map(monthDay_to_int).fillna(15).astype(np.int32)

##this finds columns which only have one value
data_temp =data.fillna(-999) #fill a temporary datafile with -999 dummy value
y = data_temp.describe().iloc[2]!=0 #get cols with std ==0
cols = y[y==True].index
data = data[list(cols)]
gc.collect()

# approximate, if there are more than 20 unique values in column, column is quantitive,else qualitiave
num_of_unqiue_entries_per_column = data.apply(lambda x : len(pd.unique(x)))
quant_var = num_of_unqiue_entries_per_column[num_of_unqiue_entries_per_column>=20].index
quant_var = [x for x in quant_var if '_STATE' != x]
qual_var = num_of_unqiue_entries_per_column[num_of_unqiue_entries_per_column<20].index
data[qual_var] = data[qual_var].fillna(-999).astype(np.int32)
# not to fill quant_var because corelation will get affected by dummy var
#data[quant_var] = data[quant_var].fillna(-999).astype(np.float32)

# **2. Sample 6000 datapoints (data already collected by client)**

In [0]:
np.random.seed(1)
sample = data.sample(n=6000,replace=True)
sample['ori_index'] = sample.index
sample  = sample.reset_index(drop=True)
data_ori = data.copy(deep=True)

# **3. Cluster questions by correlation matrix and mutual information (normalized) and split them into 15 blocks**

In [0]:
from scipy.cluster.hierarchy import dendrogram, linkage 
import partial_corr #import partial_corr
for i in quant_var:
    sample[i] = sample[i].fillna(np.mean(sample[i].dropna()))
corr = np.abs(sample[quant_var].corr().fillna(0).values)
#corr= partial_corr.partial_corr(data[quant_var].fillna(0).values)

#corr=data[quant_var].corr() #using population correlation coeficient


       
   
from sklearn.metrics import normalized_mutual_info_score
mutual_info = np.zeros((len(qual_var),len(qual_var)))+1
for i in range(len(qual_var)):
    for j in range(i+1,len(qual_var)):
        mutual_info[i,j] = normalized_mutual_info_score(sample[qual_var[i]],sample[qual_var[j]])
        mutual_info[j,i] = mutual_info[i,j]
   
if True:
    num_block = 15
    import matplotlib.pyplot as plt
    plt.close()
    linked = linkage(corr[:,:], 'single')
    dendro  = dendrogram(linked, 
                orientation='top',
                labels=quant_var[:],
                distance_sort='descending',
                show_leaf_counts=True,color_threshold=15)
    import matplotlib.pyplot as plt
    plt.tight_layout(rect=[0.025, 0.025, .94, .94]);
    plt.title('Hierarchical Clustering of\nQuantitive Variables from abs|correlation matrix|')
    plt.ylabel('Distance');plt.xlabel('Quantitive Variables')
    plt.savefig('variables_clustering_by_corr_coef.png',dpi=300);plt.close()
    a=plt.imshow(corr[np.array(dendro['leaves']),:][:,np.array(dendro['leaves'])]);plt.colorbar(a);plt.close()
    df_corr = pd.DataFrame(corr,columns=quant_var)
    df_corr.to_csv('corr.csv',index=0)
    question_quant = {}
    for block in range(num_block):
        question_quant[block] = []
    for i in range(len(dendro['ivl'])): #give each question by splitting clusters, except state which will be treated differentlys
        question_quant[i%num_block] += [dendro['ivl'][i],]
    #quanl var
    plt.close()
    plt.figure(figsize=(10,8))
    linked = linkage(mutual_info[:,:], 'single')
    dendro  = dendrogram(linked,
                orientation='top',
                labels=qual_var,
                distance_sort='descending',
                show_leaf_counts=True,color_threshold=15)
    import matplotlib.pyplot as plt
    plt.tight_layout(rect=[0.025, 0.025, .95, .92]);
    plt.title('Hierarchical Clustering of \ualitative Variables\n from normalized mutual information matrix',fontsize=24)
    plt.ylabel('Distance',fontsize=24);plt.xlabel('Qualitative Variables',fontsize=24)
    plt.xticks(range(0,len(dendro['ivl'])*10,10),dendro['ivl'],fontsize=2.5)
    plt.savefig('Qual_variables_clustering_by_corr_coef.png',dpi=500);
    df_mutual_info = pd.DataFrame(mutual_info,columns=qual_var)
    df_mutual_info.to_csv('mutual_info.csv',index=0)
    question_qual = {}
    for block in range(num_block):
        question_qual[block] = []
    for i in range(len(dendro['ivl'])):
        question_qual[i%num_block] += [dendro['ivl'][i],]

# **4. Defining my own MICE(multiple imputation with chained equations) and KL-divergence formula**

In [0]:
def MICE(df,qual_var,quant_var,iterations=10):
    from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
    from scipy.stats import mode
    # imputes dataframe using descision tree regressor/classifier. 
    #df =pandas dataframe
    #qual_var = categorical var list
    #qual_var= qualitative var list
    #n iterations
    '''NOTE:
    MISSING values due to split assigned = -99
    Missing values due to interviewee = -999,
     filling the -999 with the mode and mean respectively for qual and quant variable '''
#if True:
#    df = sample60k.copy(deep=True)
    missingid = (df==-99)  
    for i in qual_var : #ignore warning below, function is just about to be deprecated
        df = df.set_value(missingid[missingid[i]==True].index,i,mode(df[df[i]!=-99][i])[0][0])
    for i in quant_var :
        df = df.set_value(missingid[missingid[i]==True].index,i,np.mean(df[(df[i]!=-99)*(df[i]!=-999)][i]))
    df = df.fillna(-999) #some bug in code, this is a work around
    df_mean_mode_inpute =df.copy(deep=True)
    
    for n in range(iterations):
        print (n)
        for i in quant_var[:] :
            model = DecisionTreeRegressor(min_samples_split=30)
            feat = [x for x in quant_var if x != i] + list(qual_var)
            missing_index = missingid[missingid[i]==True].index
            present_index = missingid[missingid[i]==False].index
            model.fit(df.iloc[present_index][feat],df.iloc[present_index][i])
            missing_predictions = model.predict(df.iloc[missing_index][feat])
            df = df.set_value(missing_index,i,missing_predictions) #fill in missing values 
        for i in qual_var[:] :
            model = DecisionTreeClassifier(min_samples_split=30)
            feat = [x for x in qual_var if x != i] + list(quant_var)
            missing_index = missingid[missingid[i]==True].index
            present_index = missingid[missingid[i]==False].index
            model.fit(df.iloc[present_index][feat],df.iloc[present_index][i])
            missing_predictions = model.predict(df.iloc[missing_index][feat])
            df = df.set_value(missing_index,i,missing_predictions)
        gc.collect()
    print ('finished running with imputed values')
    return df,df_mean_mode_inpute

def KL_divergence(data1,data2,feat):
    a = Counter(data1[feat]) #get discrete discrtirbution in terms of counts
    b = Counter(data2[feat])
    #normalzie the dsitribution, and fill in empty entries with small dummy number =0.01
    for j in np.unique(a.keys()+b.keys()):
        if j in a.keys():
            a[j] = a[j]*1./len(data1)
        else: a[j] = 0.01
        if j in b.keys():
            b[j] = b[j]*1./len(data2)
        else: b[j] = 0.01    
    KL_div = 0
    #print a,b
    for j in np.unique(a.keys()+b.keys()):
        KL_div += (a[j])*np.log(a[j]/b[j])
    return KL_div 

# **5. Run experiments across different sample size/question block size and compare the kl-divergence and root-meansquare error (normalized by standard deviation)**

dummary variables : 
-999 are empty responses given 
,-99 are questions not attempted due to split questionair design


In [0]:
if True:
    data[quant_var] = data[quant_var].fillna(-999).astype(np.float32)
    data = data.fillna(-999)
    result_MICE= {}
    samplesize=6000
    np.random.seed(5220)
    for samplesize in [6000,12000,25000,50000,100000][:]:
        sample60k = data.sample(n=samplesize,replace=True)
        sample60k['ori_index'] = sample60k.index
        sample60k = sample60k.sort_values('_STATE').reset_index(drop=True) #sort by state, to strafiy sampling by state
        Combinations_nC3 = list(combinations(range(num_block),3)) #nC3 combinations of the number of blocks. 
        dictt_interger_to_combinations = {}
        np.random.shuffle(Combinations_nC3)
        for i in range(len(Combinations_nC3)):
            dictt_interger_to_combinations[i] = Combinations_nC3[i]
        sample60k['block'] = (sample60k.index%len(Combinations_nC3))
        sample60k['block']=sample60k['block'].map(dictt_interger_to_combinations)
        sample60k_ori = sample60k.copy(deep=True)#for validation, 'correct' answer
        sample60k[sample60k.keys()[:-2]] = sample60k[sample60k.keys()[:-2]]*0-99 #Initalize datafile with -99, where -99 denotes missing value due to survey split. 
        sample60k['_STATE'] = sample60k_ori ['_STATE']
        sample60k['ori_index'] = sample60k_ori['ori_index']
        sample60k['block'] = sample60k_ori ['block']
        for blockcomb in Combinations_nC3:
            questions_answered = []
            for block in blockcomb : #blockcomb is tuple
                questions_answered += [x for x in  question_quant[block]]
                questions_answered += [x for x in  question_qual[block]]
            block_index = sample60k[sample60k['block']==blockcomb].index
            sample60k = sample60k.set_value(block_index,
                                             questions_answered,
                                             sample60k_ori[sample60k_ori['block']==blockcomb][questions_answered])
        imputed60k,imputed60k_mean_mode = MICE(df = sample60k.copy(deep=True),qual_var = qual_var,quant_var = quant_var,iterations=10) 
        quant_result = np.mean((imputed60k[quant_var]-sample60k_ori[quant_var])/np.std(sample60k_ori[quant_var],0)**2,0)
        qual_result = map(lambda x : (x,KL_divergence(imputed60k,data,x)), qual_var[:])
        result_MICE[(samplesize,num_block)]= (quant_result,qual_result)

# 7 gradient based methods (integrated gradients and concept activation)

In [0]:
link = 'https://wikimedia.org/api/rest_v1/media/math/render/svg/75fed3df69952839e20426827b30869b7f6515ac'
from IPython.display import Image
from IPython.core.display import HTML
Image(url= './presentation/Integrated_grad_formula.png', width=800, height=500)

In [0]:
# integrated gradients
sess = K.get_session() 
model.load_weights('mnist.h5')
from matplotlib.colors import LinearSegmentedColormap
tr_id = 359
input_X = X_val[tr_id:tr_id+1]
input_X0 = input_X*0 + np.min(input_X)
grad = np.zeros(input_X.shape)
pred = model.predict(input_X)[0]
print map(lambda x : int(x*100),pred)
m = 50.
vary_pic = np.zeros((28*28,28,28,1))
counter = 0
for i in range(28):
    for j in range(28):
        vary_pic[counter,i,j]=1 
        counter += 1

for i in range(10):
    grad = np.zeros(input_X.shape)
    if i ==0:
        plt.imshow(input_X[0,:,:,0]);plt.show()
        plt.show()
        f,ax = plt.subplots(2,5,figsize=(18,8))
    if True:#i == np.argmax(pred):
        outputTensor =model.layers[-2].output[:,i] # (?,)
        listOfVariableTensors = model.inputs[0] # (?,28,28,1)
        gradients = K.gradients(outputTensor, listOfVariableTensors)
        for num in range(50):
            grad  += (input_X-input_X0)*sess.run(gradients, feed_dict= {model.input:  input_X0 + (num/m)*(input_X-input_X0)})[0] 
            if num%50==49:
                per = (np.max(grad[0,:,:,0]/m))/(np.max(grad[0,:,:,0]/m)-np.min(grad[0,:,:,0]/m))
                cdict1 = {'red':  ((0, 0, 0),   # <- at 0.0, the red component is 0
                                   (1-per, 1, 1.0),   # <- at 0.5, the red component is 1
                                   (1, 1.0, 1.0)),  # <- at 1.0, the red component is 0

                         'green': ((0, 0.0, 0.0),   # <- etc.
                                   (1-per, 1.0, 1.0),
                                   (1, 0.0, 0.0)),

                         'blue':  ((0, 1.0, 1.0),
                                   (1-per, 1.0, 1.0),
                                   (1, 0, 0))
                         }
                bwr=LinearSegmentedColormap('bwr', cdict1)
                cax = ax[i//5,(i-5)%5].imshow(grad[0,:,:,0]/m,cmap=bwr)
                cb = plt.colorbar(cax, cmap=plt.cm.bwr,ax=ax[i//5,(i-5)%5],
                          orientation="horizontal", aspect=70) 
                ax[i//5,(i-5)%5].set_title(str(i)+'_'+str(map(lambda x : int(x*100),pred)[i]))
plt.show()

In [0]:
# concept actiation vectors, concept used is diagonal
# MAKING diagonal images
import cv2
digonal_sample = np.zeros((3000,28,28))
input_X0 = input_X[0,:,:,0]*0
for i in range(0,28,1):
    for j in range(0,28,1):
        if i in [j+1,j-1,j,j+2,j-2]: #thickness of two
            input_X0[i,j] = 1

def make_strip_samples(start,end,input_X0,digonal_sample):
    for sample in range(start,end):
        leng, = np.random.choice(range(10,20),1)
        offsetX, =  np.random.choice(range(28-leng),1)
        offsetY, =  np.random.choice(range(28-leng),1)
        sub_pic = np.copy(input_X0[:leng,:leng])
        digonal_sample[sample,offsetX:offsetX+leng,offsetY:offsetY+leng] = sub_pic
    return digonal_sample
    
digonal_sample = make_strip_samples(0,1000,cv2.GaussianBlur(input_X0,(5,5),0),digonal_sample)


input_X0 = input_X[0,:,:,0]*0
for i in range(0,28,1):
    for j in range(0,28,1):
        if i in [j+1,j,j-1]:
            input_X0[i,j] = 1
digonal_sample = make_strip_samples(1000,2000,cv2.GaussianBlur(input_X0,(5,5),0),digonal_sample)


input_X0 = input_X[0,:,:,0]*0
for i in range(0,28,1):
    for j in range(0,28,1):
        if i in [j+1,j-1,j,j+2]:
            input_X0[i,j] = 1

digonal_sample = make_strip_samples(2000,3000,cv2.GaussianBlur(input_X0,(5,5),0),digonal_sample)
digonal_sample = digonal_sample[:,:,::-1]
if True:
    plt.imshow(digonal_sample[0]);plt.title('sample of diag line');plt.show()
    plt.imshow(digonal_sample[1000]);plt.title('sample of diag line');plt.show()
    plt.imshow(digonal_sample[2301]);plt.title('sample of diag line');plt.show()
digonal_sample = np.expand_dims(digonal_sample,-1)

In [0]:
# USING model.layers[4] output, the directional derivative can be from ANY layer
model2 = keras.models.Model(model.input,map(lambda x : x.output,model.layers)[4])
model2.summary()
pred_diag = np.mean(model2.predict(digonal_sample,batch_size =512//2),(1,2)) #mean poolng of layer from 2D to 1D
random_noise = 1.*np.random.randint(0,100,size=X_train[::18].shape)/100
pred_nondiag = np.mean(model2.predict(random_noise,batch_size =512//2),(1,2)) #mean poolng of layer from 2D to 1D
pred_X = np.concatenate([digonal_sample,random_noise],0)
print pred_diag.shape,pred_nondiag.shape,pred_X.shape

In [0]:
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
sess = K.get_session() 
model.load_weights('mnist.h5')

if True:
    # train SVM from outputs from model layer
    pred_diag_unwrap = pred_diag
    pred_nondiag_unwrap = pred_nondiag
    pred_comb = np.concatenate([pred_diag_unwrap,pred_nondiag_unwrap],0)
    labels = np.array([1,]*3000 + [0,]*3056) 
    svm_model = LinearSVC().fit(pred_comb,labels)
    preds = svm_model.predict(pred_comb)
    print 'svm confusion matrix:\n',confusion_matrix(labels,preds)
    support_vec = svm_model.coef_
    for output in range(10):
        outputTensor =model.layers[-2].output[:,output] # (?,)
        listOfVariableTensors = map(lambda x : x.output,model.layers)[4]# (?,28,28,1)
        gradients = K.gradients(outputTensor, listOfVariableTensors)
        grad  = sess.run(gradients, feed_dict= {model.input:  X_train[:600]})[0][:,:,:,:] # 600,14,14,64
        grad = np.mean(grad,(1,2)) #mean pooling of gradients from 2D to 1D, # 600，1,1,64
        grad = np.reshape(grad,(600,64)) #reshape gradient to 64 vector
        grad2 = np.matmul(grad,svm_model.coef_.T/np.sum(svm_model.coef_.T**2)**.5) # get directional derivative
        print 'Mean directional derivative of mnist at logit :',output
        print np.round(np.mean(grad2),2)
        val2 = np.matmul(grad,svm_model.coef_.T/np.sum(svm_model.coef_.T**2)**.5)
        plt.plot(output,np.mean(grad2),'ro')
        plt.text(output+0.1,np.mean(grad2)+0.02,str(output),fontsize=30)

plt.ylabel('avg directional derivative',fontsize=30)
plt.xlabel('digit',fontsize=30)

plt.show()
